In [26]:
:dep ndarray = "0.15.6"
:dep ndarray-rand = "0.14.0"
:dep showata = { version = "0.3.2", features=["show_ndarray"]}
:dep polars = { version = "0.29.0", features=["ndarray"]}

In [3]:
use ndarray::{Array, Array2, stack, concatenate};
use ndarray_rand::RandomExt;
use ndarray_rand::rand_distr::Uniform;
use ndarray::prelude::*;

In [174]:
use std::f64;

#[derive(Debug)]
enum ActivationFunction {
    Tanh,
}

impl ActivationFunction {
    fn activate(&self, x: f64) -> f64 {
        match self {
            ActivationFunction::Tanh => self.tanh(x),
        }
    }
    
    fn derivative(&self, x: f64) -> f64 {
        match self {
            ActivationFunction::Tanh => self.tanh_derivative(x),
        }
    }
    
    fn tanh(&self, x: f64) -> f64 {
        x.tanh()
    }
    
    fn tanh_derivative(&self, x: f64) -> f64 {
        1.0 - self.tanh(x).powi(2)
    }
}



In [609]:
#[derive(Debug)]
struct Layer {
    len_inputs: usize,
    neurons: usize,
    function: ActivationFunction,
    weights: Array2<f64>,
    input: Option<Array2<f64>>,
    net: Option<Array2<f64>>,
    output: Option<Array2<f64>>,
    idx: Option<usize>,
}

impl Layer {
    fn new(len_inputs: usize, neurons: usize, function: ActivationFunction) -> Self {
        let shape = (neurons, len_inputs + 1);
        let weights = Array::random(shape, Uniform::new(-0.5, 0.5));

        Layer {
            len_inputs: len_inputs,
            neurons: neurons,
            function: function,
            weights: weights,
            input: None,
            net: None,
            output: None,
            idx: None,
        }
    }

    fn forward(&mut self, layer_input: &Array2<f64>) -> Array2<f64> {
        self.input = Some(layer_input.clone());
        self.net = Some(self.input.as_ref().unwrap().dot(&self.weights.t()));
        self.output = Some(self.net.as_ref().unwrap().mapv(|x| self.function.activate(x)));
        self.output.clone().unwrap()
    }

    
    fn backward(
        &mut self, 
        alpha: f64,
        last: bool,
        previous_delta: Option<&Array2<f64>>,
        previous_weight: Option<&Array2<f64>>,
        error: Option<&Array2<f64>>
    ) -> (Array2<f64>, Array2<f64>) {
        
        let delta = if last {
            let error = error.unwrap();
            error * self.net.as_ref().unwrap().mapv(|x| self.function.derivative(x))
        } else {
            let previous_delta = previous_delta.unwrap();
            let previous_weight = previous_weight.unwrap();
            let delta = previous_delta.dot(previous_weight).slice(s![.., 1..]).to_owned();
            delta * self.net.as_ref().unwrap().mapv(|x| self.function.derivative(x))
        };
        
        
        self.weights = delta.t().dot(self.input.as_ref().unwrap()) * alpha + &self.weights;
        
        (delta, self.weights.to_owned())
    }



    fn set_idx(&mut self, idx: usize) {
        self.idx = Some(idx);
    }
}

In [616]:
#[derive(Debug)]
struct NeuralNetwork {
    layers: Vec<Layer>,
    all_mse: Vec<f64>,
}

impl NeuralNetwork {
    fn new(mut layers: Vec<Layer>) -> Self {
        for (idx, layer) in layers.iter_mut().enumerate() {
            layer.set_idx(idx + 1);
        }
        
        NeuralNetwork {
            layers,
            all_mse: Vec::new(),
        }
    }

    fn forward(&mut self, x_input: &Array2<f64>) -> Array2<f64> {
        let mut input_layer = concatenate![Axis(1), Array::from_shape_vec((1, 1), vec![1.0]).unwrap(), x_input.clone()];
        let mut output: Array2<f64> = Array::zeros((0, 0));

        for layer in &mut self.layers {
            output = layer.forward(&input_layer);
            input_layer = concatenate![Axis(1), Array::from_shape_vec((1, 1), vec![1.0]).unwrap(), output];
        }

        output
    }

    fn backward(&mut self, alpha: f64, error: &Array2<f64>) {
        let mut previous_delta = None;
        let mut previous_weight = None;
        let mut last = true;
        
        for layer in self.layers.iter_mut().rev() {
            let (delta, weights) = layer.backward(alpha, last, previous_delta.as_ref(), previous_weight.as_ref(), Some(&error));
            last = false;
            previous_delta = Some(delta);
            previous_weight = Some(weights);
        }
    }

    fn predict(&mut self, x: &Array2<f64>) -> Array2<f64> {
        self.forward(x)
    }
}

In [617]:
let weights_1 = array![[0.2, 0.4, 0.5],[0.3, 0.6, 0.7],[0.4, 0.8, 0.3]];
let weights_2 = array![[-0.7, 0.6, 0.2, 0.7],[-0.3, 0.7, 0.2, 0.8]];
let weights_3 = array![[0.1, 0.8, 0.5]];


let mut nn = NeuralNetwork::new(vec![
    Layer::new(2, 3, ActivationFunction::Tanh),
    Layer::new(3, 2, ActivationFunction::Tanh),
    Layer::new(2, 1, ActivationFunction::Tanh),
]);

nn.layers[0].weights = weights_1;
nn.layers[1].weights = weights_2;
nn.layers[2].weights = weights_3;

let x_inputs = array![[0.3, 0.7]];
let out = nn.forward(&x_inputs);
out


[[0.5763660324621392]], shape=[1, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2

In [618]:
nn.backward(0.05, &array![[-2.0]]);
nn.predict(&x_inputs)

[[0.403346832835286]], shape=[1, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2

In [43]:
use polars::prelude::*;
let file_path = "train.csv";

    // Ler o arquivo CSV para um DataFrame
let df: DataFrame = CsvReader::from_path(file_path)
    .unwrap()
    .infer_schema(None)
    .has_header(true)
    .finish()
    .unwrap();

// Exibir o DataFrame

In [110]:
:dep ndarray-csv = {version = "0.4.1"}
:dep csv

In [163]:
let s = df.column("label").unwrap().to_owned();
let s = s.i64().unwrap().to_owned();
let s = s.to_ndarray().unwrap().to_owned();
let s = s.into_shape((42000,1)).unwrap().mapv(|x| x as f64);;
s

[[1.0],
 [0.0],
 [1.0],
 [4.0],
 [0.0],
 ...,
 [0.0],
 [1.0],
 [7.0],
 [6.0],
 [9.0]], shape=[42000, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2

In [170]:
df.get_row(0).unwrap().to_vec()

Error: no method named `to_vec` found for struct `Row` in the current scope